In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
vel=np.loadtxt('vel.txt')/1000 # v_LOS in km s^-1

In [ ]:
a=np.loadtxt('21cmsurvey.txt')

In [ ]:
long=a[0]
long.shape # galactic longitude in deg

In [ ]:
spgr=a[1:] # all spectra in Tb: brightness temperature (K)

In [ ]:
spgr.shape

In [ ]:
plt.figure(figsize=(5,5))
plt.imshow(spgr,extent=[long[-1],long[0],vel[-1],vel[0]],aspect='auto')
plt.xlabel('long (deg)')
plt.ylabel('v_LOS (km/s)')

In [ ]:
plt.plot(vel,spgr[:,0])
plt.xlabel('v (km/s)')
plt.ylabel('Tb (K)')

In [ ]:
def gauss(x, *p):
    A, mu, sigma = p
    return A*np.exp(-(x-mu)**2/(2.*sigma**2))

In [ ]:
i=0
x=vel
y=spgr[:,i]

In [ ]:
g1=gauss(x, y.max(),x[np.argmax(y)],1)
hist_fit = g1

plt.plot(x, y, label='Test data')
plt.plot(x, hist_fit, label='Fitted data')
plt.title('long = '+str(long[i])[:6]+'º')
plt.xlabel('v (km/s)')
plt.ylabel('Tb (K)')

In [ ]:
lon=long[0]*np.pi/180
v_sun=225
R=8*(v_sun*np.sin(lon))/(x[np.argmax(y)]+v_sun*np.sin(lon))
R # in kpc

In [ ]:
from lmfit.models import GaussianModel

In [ ]:
gauss1 = GaussianModel(prefix='g1_')
pars=gauss1.make_params()

pars['g1_center'].set(value=-30, min=-45, max=-20)
pars['g1_sigma'].set(value=2, min=1,max=10)
pars['g1_amplitude'].set(value=1000, min=20,max=3000)
                       

In [ ]:
mod = gauss1

In [ ]:
out = mod.fit(y, pars, x=x)
print(out.fit_report(min_correl=0.5))

In [ ]:
comps = out.eval_components(x=x)
plt.plot(x, y, 'b')
plt.plot(x, comps['g1_'], 'g--', label='Gaussian component 1')
plt.legend(loc='best')

In [ ]:
dv=vel[0]-vel[1]
integ=np.trapz(comps['g1_'], dx=dv)
NH=1.82e18*integ